In [1]:
pip install praw

In [2]:
import praw
import pandas as pd
# Reddit credentials, password stored in .env 
# PRAW setup and login goes here, omitted for privacy

In [3]:
# set up some sort of corpus of keywords to snag specific reddit entries
all_locs = pd.read_json('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations-v2.json')
all_locs = pd.json_normalize(all_locs['data'])
all_locs = all_locs.drop(columns=['edit_at','id'])
def cleanlinks(json):
    links_out = []
    for link in json:
        links_out.append(link['url'])
    return links_out
all_locs['links'] = all_locs['links'].apply(cleanlinks)
all_locs['date'] = pd.to_datetime(all_locs['date'],format='%Y-%m-%d')
all_tags = all_locs['tags'].copy()
tags = set()
for taglist in all_tags:
  for tag in taglist:
    if tag not in tags:
      tags.add(tag)
# manually remove needless tags
print(tags)
tags.discard('')
tags.discard('medic')
tags.discard('bike')
tags.discard('non-protest')
tags.discard('pregnant')
tags.discard('lgbtq+')
tags.discard('racial-profiling')
tags.discard('legal-observer')
tags.discard('tear-gas-canister')
tags.discard('politician')
tags.discard('incitement')
tags.discard('homeless')
tags.discard('shoot')
tags.discard('strike')
tags.discard('elderly')
tags.discard('vehicle')
tags.discard('inhumane-treatment')
tags.discard('journalist')
tags.discard('throw')
tags.discard('explosive')
tags.discard('threaten')
tags.discard('horse')
tags.discard('shove')
tags.discard('child')
tags.discard('shield')
tags.discard('dog')
tags.discard('knee')
tags.discard('protester')
tags.discard('gun')
tags.discard('conceal')
tags.discard('bystander')
tags.discard('grab')
tags.discard('push')
tags.discard('zip-tie')
tags.discard('spray')
tags.discard('drive')
tags.discard('person-with-disability')
tags.discard('celebrity')
tags.discard('projectile')
tags.discard('beat')
newtags = set()
for tag in tags:
  tag = newtags.add(tag.replace('-',' '))
tags = newtags
print(tags)

{'marking-round', '', 'knee-on-neck', 'strike', 'bike', 'non-protest', 'lgbtq+', 'conceal', 'gun', 'gas', 'live-round', 'tear-gas', 'knee', 'zip-tie', 'pepper-spray', 'death', 'horse', 'rubber-bullet', 'tackle', 'arrest', 'pepper-ball', 'shove', 'inhumane-treatment', 'person-with-disability', 'tase', 'beat', 'baton', 'elderly', 'mace', 'punch', 'shield', 'headlock', 'legal-observer', 'politician', 'homeless', 'kick', 'shoot', 'medic', 'protester', 'bystander', 'push', 'child', 'projectile', 'stun-grenade', 'wooden-bullet', 'journalist', 'hide-badge', 'abuse-of-power', 'incitement', 'celebrity', 'foam-bullet', 'vehicle', 'drive', 'choke', 'body-cam', 'dog', 'threaten', 'explosive', 'racial-profiling', 'throw', 'property-destruction', 'spray', 'tear-gas-canister', 'pregnant', 'lrad', 'taser', 'grab', 'bean-bag'}
{'property destruction', 'marking round', 'rubber bullet', 'pepper spray', 'gas', 'death', 'tear gas', 'hide badge', 'tackle', 'arrest', 'tase', 'pepper ball', 'baton', 'knee on 

In [6]:
# Grabbing 100 hottest posts on Reddit at the moment. Will filter for police use of force later

data = []

# other possible subreddits: publicfreakout, allcopnodonut
for submission in reddit.subreddit("news").hot(limit=1000):
  data.append([submission.id, submission.title, submission.score, submission.subreddit, submission.url, 
               submission.num_comments, submission.selftext, submission.created])

# We'll need a way to get coordinates for a given post, before we include that in df
col_names = ['id', 'title', 'score', 'subreddit', 'url', 
             'num_comments', 'text', 'created']
df = pd.DataFrame(data, columns=col_names)

df.head()

,id,title,score,subreddit,url,num_comments,text,created
0,icq91v,Pennsylvania Among States To Sue Over Changes ...,14873,news,https://levittownnow.com/2020/08/18/pa-among-s...,392,,1.597881e+09
1,icsg56,Mark Zuckerberg Is Despoiling a Tiny Coastal V...,4197,news,https://www.wweek.com/news/2020/08/19/mark-zuc...,325,,1.597888e+09
2,icm712,New Mexico sues over US Postal Service changes.,69055,news,https://www.kob.com/new-mexico-news/new-mexico...,2161,,1.597867e+09
3,icncj5,Man Accused Of Assaulting Sesame Place Worker ...,4302,news,http://levittownnow.com/2020/08/19/man-accused...,400,,1.597871e+09
4,icu2tz,QAnon groups hit by Facebook crack down,1189,news,https://www.nbcnews.com/tech/tech-news/qanon-g...,310,,1.597893e+09


In [8]:
import requests
from bs4 import BeautifulSoup
import re
# get the url of the reddit post
for url in df['url']:
  # get the HTML from the url
  try:
    r = requests.get(url, timeout=10)
  except:
    continue
  soup = BeautifulSoup(r.text)
  # get tags from metadata for the site
  sitetags = set()
  for meta in soup.find_all('meta'):
    if meta is None:
      continue
    meta = str(meta)
    meta = meta.lower()
    meta = re.sub('[\W_]+',' ', meta)
    for tag in str(meta).split():
      sitetags.add(tag)
  tags_final = sitetags & tags
  if tags_final:
    # some matches found, print title and matched tags
    print(soup.title.text.strip(), tags_final)

4 teens, including juvenile, arrested in connection with Raleigh girl’s murder outside Bojangles’ {'death'}
UConn students evicted from dorms for holding pandemic party as schools grapple with COVID-19 crisis {'death'}
Police searching for suspect who sucker-punched elderly sidewalk sweeper – Boston News, Weather, Sports | WHDH 7News {'punch'}
Victims Want Golden State Killer Sent to Toughest  Prison – NBC 7 San Diego {'death'}
Minneapolis officer's attorney alleges George Floyd overdosed on fentanyl, says charges should be dropped {'arrest', 'death'}
'Should've ducked': Detention officer suspended over post on boy who was shot to death {'death'}
Phoenix police investigating the death of a man in their custody - CNN {'death'}
InfoWars’ Millie Weaver, boyfriend and brother threw mother to the ground and took her cellphone, police report says - cleveland.com {'arrest'}
Rolling Blackouts in California Have Power Experts Stumped - The New York Times {'gas'}
OPEC+ presses for compliance wit